In [24]:
import PyPDF2
from PyPDF2 import PdfReader
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

from IPython.display import display, HTML

from util import local_settings

display(HTML(f"""
✅ Libraries loaded successfully <br>
✅ OpenAI Key loaded (...{local_settings.OPENAI_API_KEY[10:-15]}...)
"""))

In [25]:
pdf_data = PdfReader("C:\\Users\\catio\\Downloads\\meds.pdf")

pdf_text = ""

for i, page in enumerate(pdf_data.pages):
    text = page.extract_text()
    if text:
        pdf_text += text

print(len(pdf_text))

842875


In [28]:
llm = OpenAI(model_name ="gpt-3.5-turbo-instruct")

medicine = "paracetamol"

prompt = f" When should I not take {medicine}? Answer 'I don't know' if you don't know."

response = llm(prompt, temperature = 0)

print("----- response -----")
print(response.replace("Langchain", ""))

----- response -----


I don't know.


In [29]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 400,
    chunk_overlap = 100
)

final_data = text_splitter.split_text(pdf_text)

print(f"""
    # of Chunks: {len(final_data)}
    Chunk 0: {final_data[0]}
    Chunk 1: {final_data[1]}
""")

Created a chunk of size 543, which is longer than the specified 400
Created a chunk of size 502, which is longer than the specified 400
Created a chunk of size 526, which is longer than the specified 400
Created a chunk of size 486, which is longer than the specified 400
Created a chunk of size 561, which is longer than the specified 400
Created a chunk of size 702, which is longer than the specified 400
Created a chunk of size 611, which is longer than the specified 400
Created a chunk of size 402, which is longer than the specified 400
Created a chunk of size 561, which is longer than the specified 400
Created a chunk of size 443, which is longer than the specified 400
Created a chunk of size 504, which is longer than the specified 400
Created a chunk of size 429, which is longer than the specified 400
Created a chunk of size 448, which is longer than the specified 400
Created a chunk of size 653, which is longer than the specified 400
Created a chunk of size 427, which is longer tha


    # of Chunks: 2810
    Chunk 0: APROVADO EM 
 06-04-2023  INFARMED 
 
 Folheto informativo: Informação para o doente 
 
Varfine 5 mg comprimidos 
varfarina 
  
Leia com atenção todo este folheto antes de começar a tomar este medicamento, pois
    Chunk 1: contém informação importante para si. - Conserve este folheto. Pode ter necessidade de o ler novamente.  - Caso ainda tenha dúvidas, fale com o seu médico ou farmacêutico. - Este medicamento foi receitado apenas para si. Não deve dá-lo a outros. O medicamento pode ser-lhes prejudicial mesmo que apresentem os mesmos sinais de doença. - Se tiver quaisquer efeitos indesejáveis, incluindo possíveis efeitos indesejáveis não indicados neste folheto, fale com o seu médico ou farmacêutico ou enfermeiro. Ver secção 4.  O que contém este folheto:



In [30]:
embeddings = OpenAIEmbeddings()
document_searcher = FAISS.from_texts(final_data, embeddings)
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [41]:
medicine = "clonixina"

prompt = f" Que medicamentos são contra indicados com {medicine}? Se não souberes diz para consultar o médico"

docs =  document_searcher.similarity_search(prompt)
result = chain.run(input_documents=docs, question=prompt)

print("--- 🤖 RESULT ---")
print(result)

--- 🤖 RESULT ---
 Outros AINEs (Anti-Inflamatórios Não Esteroides), anticoagulantes e antiagregantes orais (heparina e heparina de baixo peso molecular HBPM).
